1. У меня победила встроенная токенизация, у нее совсем немного выше и пресижен, и реколл, и следовательно общая точность.

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from razdel import tokenize
from IPython.display import Image
from IPython.core.display import HTML 

data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [13]:
#Векторайзер с встроенным токенизатором
vectorizer1 = TfidfVectorizer(min_df=10, max_df=0.3)
X1 = vectorizer1.fit_transform(train.comment)
X_test1 = vectorizer1.transform(test.comment) 

y1 = train.toxic.values
y_test1 = test.toxic.values


In [14]:
X1.shape, X_test1.shape

((12970, 3364), (1442, 3364))

In [15]:
#Векторайзер с токенизатором razdel
def razdel_tokenizer(text):
    return [token.text for token in tokenize(text)]

vectorizer2 = TfidfVectorizer(tokenizer=razdel_tokenizer, min_df=10, max_df=0.3)
X2 = vectorizer2.fit_transform(train.comment)
X_test2 = vectorizer2.transform(test.comment) 

y2 = train.toxic.values
y_test2 = test.toxic.values


C:\Users\sofya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [16]:
X2.shape, X_test2.shape

((12970, 3435), (1442, 3435))

In [17]:
#Обучение с встроенной токенизацией
clf1 = LogisticRegression(C=0.1, class_weight='balanced')
clf1.fit(X1, y1)
preds = clf1.predict(X_test1)
print(classification_report(y_test1, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.82      0.85       985
         1.0       0.66      0.76      0.71       457

    accuracy                           0.80      1442
   macro avg       0.77      0.79      0.78      1442
weighted avg       0.81      0.80      0.80      1442



In [18]:
#Обучение с токенизацией razdel
clf2 = LogisticRegression(C=0.1, class_weight='balanced')
clf2.fit(X2, y2)
preds = clf2.predict(X_test2)
print(classification_report(y_test2, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.79      0.83       985
         1.0       0.63      0.77      0.69       457

    accuracy                           0.79      1442
   macro avg       0.76      0.78      0.76      1442
weighted avg       0.80      0.79      0.79      1442



2. БОльшую разницу я ощутила не столько между классификаторами, сколько между векторайзерами, с каунтвекторайзером более топовыми получаются длинные тексты, с тфидф более короткие, я так испугалась такой сильной разницы, что сделала не два, а три классификатора, чтобы убедиться, что это точно из-за векторайзеров, а не потому что я с обучением налажала))) Токсик по сути есть почти во всех выведенных предложениях, но есть сомнения насчет 10го из второго и третьего классификаторов.
   Больше совпадений получилось между вторым классификатором и третьим, но на самом деле это зависит от подборки текстов, иногда рандомфорест совсем что-то на своей волне выдает)

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
stop_words_russian = list(stopwords.words('russian'))
#понимаю, что в set читалось бы быстрее, но у меня почему-то не получилось придумать, как в векторайзер вставить сет, он ругался сильно


data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

#первая машинка
vectorizer1 = CountVectorizer(min_df=10, max_df=0.3, stop_words=stop_words_russian, lowercase=True, encoding='UTF-8')
X1 = vectorizer1.fit_transform(train.comment)
X_test1 = vectorizer1.transform(test.comment)


y1 = train.toxic.values
y_test1 = test.toxic.values


clf1 = LogisticRegression(C=0.1, class_weight='balanced')
clf1.fit(X1, y1)


preds1 = clf1.predict(X_test1)

print(classification_report(y_test1, preds1, zero_division=0))

probs1 = clf1.predict_proba(X_test1)[:, 1]
test['toxic_prob1'] = probs1

top_10_sentences1 = test.sort_values(by='toxic_prob1', ascending=False).head(10)['comment']

for i, sentence in enumerate(top_10_sentences1, 1):
    print(f"{i}. {sentence}")

#вторая машинка
vectorizer2 = TfidfVectorizer(min_df=10, max_df=0.3, stop_words=stop_words_russian, lowercase=True, encoding='UTF-8')
X2 = vectorizer2.fit_transform(train.comment)
X_test2 = vectorizer2.transform(test.comment)


y2 = train.toxic.values
y_test2 = test.toxic.values


clf2 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
clf2.fit(X2, y2)


preds2 = clf2.predict(X_test2)


print(classification_report(y_test2, preds2, zero_division=0))

probs2 = clf2.predict_proba(X_test2)[:, 1]
test['toxic_prob2'] = probs2

top_10_sentences2 = test.sort_values(by='toxic_prob2', ascending=False).head(10)['comment']

for i, sentence in enumerate(top_10_sentences2, 1):
    print(f"{i}. {sentence}")

#третья машинка
vectorizer3 = TfidfVectorizer(min_df=10, max_df=0.3, stop_words=stop_words_russian, lowercase=True, encoding='UTF-8')
X3 = vectorizer3.fit_transform(train.comment)
X_test3 = vectorizer3.transform(test.comment)


y3 = train.toxic.values
y_test3 = test.toxic.values

clf3 = MultinomialNB()
clf3.fit(X3, y3)


preds3 = clf3.predict(X_test3)


print(classification_report(y_test3, preds3, zero_division=0))

probs3 = clf3.predict_proba(X_test3)[:, 1]
test['toxic_prob3'] = probs3

top_10_sentences3 = test.sort_values(by='toxic_prob3', ascending=False).head(10)['comment']

for i, sentence in enumerate(top_10_sentences3, 1):
    print(f"{i}. {sentence}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sofya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

         0.0       0.92      0.75      0.83       959
         1.0       0.64      0.86      0.73       483

    accuracy                           0.79      1442
   macro avg       0.78      0.81      0.78      1442
weighted avg       0.82      0.79      0.80      1442

1. В Киеве на вокзале Мен було рок в 19, коли мене перший раз мав в зад хлопець рок в п д 30. Я тод перш рази став заходити на вокзал Ки в-Пасажирський в туалети - де були каб ни з д рками написи на ст нах. Так як досв ду ще не було н якого, то як знайомиться не уявляв. Сам перший природно не п дходив. А видивлявся на написи. дрочив св й член стоячи в каб нц . Хлопець був у сус дн й каб нц , в н побачив це, хитнув мен головою, запрошуючи п ти з ним. А так як н кого б льше в той момент не було, а був уже веч р, над на щось нше не було -все ж п шов за ним. У мене вже тод з явилася молофья - я вже спускав. Так як трохи ран ше ще не було, при дрочц робив це до при мног

3. Кажется, у меня что-то сломалось с топом в NB, мне кажется, такие слова должны были вообще на этапе стоп-слов улететь, но у меня уже нет сил это в 10 раз переписывать, там одно и то же получается(

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
stop_words_russian = list(stopwords.words('russian'))

data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

vectorizer = CountVectorizer(min_df=10, max_df=0.3, stop_words=stop_words_russian)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sofya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Логистическая регрессия

In [8]:
clf1 = LogisticRegression(C=0.1, class_weight='balanced')
clf1.fit(X, y)


preds1 = clf1.predict(X_test)

print(classification_report(y_test, preds1, zero_division=0))

feature_names1 = vectorizer.get_feature_names_out()

coefficients1 = clf1.coef_[0]

coefficients_df1 = pd.DataFrame({'Feature1': feature_names1, 'Coefficient1': coefficients1})

top_words1 = coefficients_df1.sort_values(by='Coefficient1', ascending=False).head(10)

print(top_words1)

              precision    recall  f1-score   support

         0.0       0.90      0.77      0.83       968
         1.0       0.64      0.83      0.73       474

    accuracy                           0.79      1442
   macro avg       0.77      0.80      0.78      1442
weighted avg       0.82      0.79      0.80      1442

     Feature1  Coefficient1
3092    хохлы      1.496368
3091   хохлов      1.458885
2833     тебе      1.252565
539     дебил      1.207219
1418    нахуй      1.074247
1811   пиздец      1.051941
2767     сука      1.031327
159     блядь      0.975234
2384  русских      0.964951
2923    тупые      0.952257


Random forest

In [11]:
clf2 = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
clf2.fit(X, y)


preds2 = clf2.predict(X_test)


print(classification_report(y_test, preds2, zero_division=0))

feature_names2 = vectorizer.get_feature_names_out()

importances2 = clf2.feature_importances_

importances_df2 = pd.DataFrame({'Feature2': feature_names2, 'Importance2': importances2})

top_words2 = importances_df2.sort_values(by='Importance2', ascending=False).head(10)

print(top_words2)

              precision    recall  f1-score   support

         0.0       0.84      0.84      0.84       968
         1.0       0.66      0.66      0.66       474

    accuracy                           0.78      1442
   macro avg       0.75      0.75      0.75      1442
weighted avg       0.78      0.78      0.78      1442

      Feature2  Importance2
1742     очень     0.009075
2833      тебе     0.008370
3092     хохлы     0.008005
3091    хохлов     0.007603
3219       это     0.007478
1418     нахуй     0.006637
1395  например     0.005924
863       знаю     0.005526
2651   спасибо     0.005054
1139       лет     0.005030


Наивная байесовская

In [9]:
clf3 = MultinomialNB()
clf3.fit(X, y)


preds3 = clf3.predict(X_test)

print(classification_report(y_test, preds3, zero_division=0))

feature_names3 = vectorizer.get_feature_names_out()

log_probabilities3 = clf3.feature_log_prob_

log_probabilities_df3 = pd.DataFrame({'Feature3': feature_names3, 'Log Probability3': log_probabilities3[1]})

top_words3 = log_probabilities_df3.sort_values(by='Log Probability3', ascending=False).head(10)

print(top_words3)

              precision    recall  f1-score   support

         0.0       0.84      0.93      0.88       968
         1.0       0.82      0.63      0.71       474

    accuracy                           0.83      1442
   macro avg       0.83      0.78      0.79      1442
weighted avg       0.83      0.83      0.82      1442

     Feature3  Log Probability3
3219      это         -3.461941
2185   просто         -4.508633
2833     тебе         -4.932117
377       всё         -5.275888
721       ещё         -5.369917
337    вообще         -5.376984
2039   почему         -5.398491
1418    нахуй         -5.538252
3092    хохлы         -5.662305
1086  которые         -5.720574


Дерево решений

In [10]:
clf4 = DecisionTreeClassifier(class_weight='balanced')  
clf4.fit(X, y)

preds4 = clf4.predict(X_test)

print(classification_report(y_test, preds1, zero_division=0))


feature_names4 = vectorizer.get_feature_names_out()

importances_tree4 = clf4.feature_importances_

importances_tree_df4 = pd.DataFrame({'Feature4': feature_names4, 'Importance4': importances_tree4})

top_words4 = importances_tree_df4.sort_values(by='Importance4', ascending=False).head(10)

print(top_words4)

              precision    recall  f1-score   support

         0.0       0.90      0.77      0.83       968
         1.0       0.64      0.83      0.73       474

    accuracy                           0.79      1442
   macro avg       0.77      0.80      0.78      1442
weighted avg       0.82      0.79      0.80      1442

      Feature4  Importance4
2833      тебе     0.012936
1742     очень     0.012603
3092     хохлы     0.011437
1418     нахуй     0.010158
3091    хохлов     0.009492
1139       лет     0.009368
3219       это     0.008314
863       знаю     0.008221
1395  например     0.007161
159      блядь     0.006100
